In [62]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Define data paths

In [3]:
data_dir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games/Data/2_eye/New')
game_dir = os.path.join(data_dir, 'game')
eyet_dir = os.path.join(data_dir, 'eyet')
mous_dir = os.path.join(data_dir, 'mous')
game_files = [os.path.join(game_dir, g) for g in os.listdir(game_dir) if g[-3:]=='csv']
eyet_files = [os.path.join(eyet_dir, e) for e in os.listdir(eyet_dir) if e[-3:]=='csv']
mous_files = [os.path.join(mous_dir, m) for m in os.listdir(mous_dir) if m[-3:]=='csv']

# Load data

In [94]:
subject_initial_map = [g[-6:-4] for g in game_files]
subject_initial_map = dict(zip(subject_initial_map, np.arange(len(subject_initial_map))))

def load_game_file(gf):
    D = pd.read_csv(gf)
    D['subject'] = gf[-6:-4]
    D['human'] = D['subject'].map(subject_initial_map)
#     D['time'] = D['time']*1000
    return D#.set_index('time')

def load_mous_file(mf):
    mfnames = ['idx', 'id', 'color', 'gi', 'mi', 'status', 'bp', 'wp', 'zet', 'rt', 'ts', 'mt', 'mx']
    D = pd.read_csv(mf, names=mfnames)
    D = D[['mt', 'mx']]
    valid = pd.notnull(D['mt'])
    m = (D.loc[valid, 'mt'] + ',').sum().split(',')[:-1]
    x = [tuple(xx.split(',')) for xx in (D.loc[valid, 'mx'] + ';').sum().split(';')][:-1]
    M = pd.DataFrame(index=m, data=x, columns=['x', 'y'])
    M['subject'] = mf[-6:-4]
    M['human'] = M['subject'].map(subject_initial_map)
    M.index = M.index.astype(int)
    return M

def load_eyet_file(ef):
    D = pd.read_csv(ef)
#     D = D.loc[D['etype']=='F']
    D['subject'] = ef[-6:-4]
    D['human'] = D['subject'].map(subject_initial_map)
    D[['start', 'end']] = (D[['start', 'end']]*1000).astype(int)
    return D[['start', 'end', 'transx', 'transy', 'human']].set_index('start')

def make_tidy(subject_index):
    e, m, g = [l[subject_index] for l in [e_list, m_list, g_list]]
    start_time = int(e.index[0])
    end_time = int(e.loc[e.index.values[-1], 'end'])
    m = m.loc[(m.index > start_time) & (m.index < end_time)]
    D = pd.DataFrame(index=np.arange(start_time, end_time, 1))
    D.loc[e.index, 'eyex'] = e['transx'].astype(float)
    D.loc[e.index, 'eyey'] = e['transy'].astype(float)
    
    D.loc[m.index, 'moux'] = m['x'].astype(float)
    D.loc[m.index, 'mouy'] = m['y'].astype(float)
    
    D.loc[g.index, 'turn'] = 100*g['gi'] + g['mi']

    return D.dropna(how='all').fillna(method='ffill')

e_list = [load_eyet_file(e) for e in eyet_files]
m_list = [load_mous_file(m) for m in mous_files]
g_list = [load_game_file(g) for g in game_files]

In [95]:
test = g_list[0]
test.loc[test.gi.isin([10, 12]), :]

,human,computer,bp,wp,response,status,subject,rt,gi,mi,color,time
9,0,0.00,000000000000000000000000000000000000,000000000000000000000000000000000000,9,eyecal,AN,0.877,10,-1,2,1.429294e+09
11,0,0.00,000000000000000000000000000000000000,000000000000000000000000000000000000,11,eyecal,AN,0.576,12,-1,2,1.429294e+09
185,0,9.01,000000000000000000000000000000000000,000000000000000000000000000000000000,13,playing,AN,2.405,10,0,0,1.429295e+09
186,0,9.01,000000000000010000000000000000000000,000000000000000000000000000000000000,30,playing,AN,0.000,10,1,1,1.429295e+09
187,0,9.01,000000000000010000000000000000000000,000000000000000000000000000000100000,14,playing,AN,1.743,10,2,0,1.429295e+09
188,0,9.01,000000000000011000000000000000000000,000000000000000000000000000000100000,15,playing,AN,0.000,10,3,1,1.429295e+09
189,0,9.01,000000000000011000000000000000000000,000000000000000100000000000000100000,23,playing,AN,7.312,10,4,0,1.429295e+09
190,0,9.01,000000000000011000000001000000000000,000000000000000100000000000000100000,3,playing,AN,0.000,10,5,1,1.429295e+09
191,0,9.01,000000000000011000000001000000000000,000100000000000100000000000000100000,5,playing,AN,4.560,10,6,0,1.429295e+09
192,0,9.01,000001000000011000000001000000000000,000100000000000100000000000000100000,32,playing,AN,0.000,10,7,1,1.429295e+09


In [84]:
t = [make_tidy(i) for i in range(len(e_list))]

KeyError: "Int64Index([0, 0], dtype='int64', name='time') not in index"

In [80]:
ext = t[0]
g = g_list[0]
ext.loc[g.index.astype(int), 'mi'] = g['mi']

KeyError: "Int64Index([1429294228, 1429294230, 1429294232, 1429294235, 1429294237,\n            1429294239, 1429294241, 1429294243, 1429294245, 1429294246,\n            ...\n            1429296278, 1429296287, 1429296299, 1429296310, 1429296314,\n            1429296321, 1429296327, 1429296337, 1429296344, 1429296350],\n           dtype='int64', name='time', length=409) not in index"

# Statistics

In [73]:
for i in range(len(e_list)):
    for c in ['x', 'y']:
        tt = t[i].loc[pd.notnull(t[i]['eye'+c])&pd.notnull(t[i]['mou'+c])]
        lr = stats.linregress(tt['eye'+c], tt['mou'+c])
        print(str(i), c, lr.rvalue, lr.pvalue)

0 x 0.539544575137 0.0
0 y 0.337695670704 0.0
1 x 0.575578813052 0.0
1 y 0.47371568511 0.0
2 x 0.49527556232 0.0
2 y 0.257638302771 0.0
3 x 0.365426359459 0.0
3 y 0.13451628313 1.69507280265e-208
4 x 0.697156472769 0.0
4 y 0.366603092381 0.0
5 x 0.507998952339 0.0
5 y 0.273826079129 0.0
6 x 0.612987042945 0.0
6 y 0.347491172302 0.0
7 x 0.377476613913 0.0
7 y 0.177729036279 4.68899310623e-251
8 x 0.426695874055 0.0
8 y 0.119043381724 9.74132026831e-128
9 x 0.619661147032 0.0
9 y 0.283710040935 0.0
